In [ ]:
#@title Install Required Packages

# On your local machine, uncomment them
# !pip install -qU torch
# !pip install -qU numpy
# !pip install -qU pandas

!pip install -qU transformers
!pip install -q regex
!pip install -q boto3

import sys

!git clone https://github.com/jessevig/bertviz bertviz_repo

if not '/content/bertviz_repo' in sys.path:
  sys.path += ['/content/bertviz_repo']

     |████████████████████████████████| 1.8MB 12.2MB/s 
     |████████████████████████████████| 890kB 58.8MB/s 
     |████████████████████████████████| 3.2MB 57.2MB/s 
     |████████████████████████████████| 133kB 12.6MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 7.2MB 29.4MB/s 
     |████████████████████████████████| 143kB 57.2MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
Cloning into 'bertviz_repo'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1294 (delta 51), reused 50 (delta 21), pack-reused 1203
Receiving objects: 100% (1294/1294), 213.58 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (814/814), done.


In [ ]:
#@title Load Packages

from transformers import AutoModelForMaskedLM, AutoTokenizer

from bertviz import head_view
from bertviz.neuron_view import show

from IPython import display

# BERT

In [ ]:
model_name_or_path = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForMaskedLM.from_pretrained(model_name_or_path, output_attentions=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_a = "The animal did not cross the street."
sentence_b = "Because it was too tired."


inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs["token_type_ids"]
input_ids = inputs["input_ids"]
attention = model(input_ids, token_type_ids=token_type_ids)[-1]

print(f"Attentoin has {len(attention)} heads")

sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist()
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
print("Tokens:", "\n".join(tokens))

Attentoin has 12 heads
Tokens: [CLS]
the
animal
did
not
cross
the
street
.
[SEP]
because
it
was
too
tired
.
[SEP]


## Head View

The attention-head view visualizes attention in one or more heads in a particular layer in the model.


**Usage**

- **Hover** over any **token** on the left/right side of the visualization to filter attention from/to that token. The colors correspond to different attention heads.
- **Double-click** on any of the **colored tiles** at the top to filter to the corresponding attention head.
- **Single-click** on any of the **colored tiles** to toggle selection of the corresponding attention head. 
- **Click** on the **Layer** drop-down to change the model layer (zero-indexed).
- The lines show the attention from each token (left) to every other token (right). Darker lines indicate higher attention weights. When multiple heads are selected, the attention weights are overlaid on one another. 


In [ ]:
display.display(display.HTML('<script src="/static/components/requirejs/require.js"></script>'))

head_view(attention, tokens, sentence_b_start, layer=0, heads=0)

# mBERT

In [ ]:
model_name_or_path = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForMaskedLM.from_pretrained(model_name_or_path, output_attentions=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentence_a = "حیوان از خیابان عبور نکرد."
sentence_b = "زیرا او خیلی خسته بود."


inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs["token_type_ids"]
input_ids = inputs["input_ids"]
attention = model(input_ids, token_type_ids=token_type_ids)[-1]

print(f"Attentoin has {len(attention)} heads")

sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist()
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
print("Tokens:", "\n".join(tokens))

Attentoin has 12 heads
Tokens: [CLS]
ح
##یو
##ان
از
خیابان
عبور
ن
##کرد
.
[SEP]
زیرا
او
خ
##یلی
خ
##سته
بود
.
[SEP]


## Head View

The attention-head view visualizes attention in one or more heads in a particular layer in the model.


**Usage**

- **Hover** over any **token** on the left/right side of the visualization to filter attention from/to that token. The colors correspond to different attention heads.
- **Double-click** on any of the **colored tiles** at the top to filter to the corresponding attention head.
- **Single-click** on any of the **colored tiles** to toggle selection of the corresponding attention head. 
- **Click** on the **Layer** drop-down to change the model layer (zero-indexed).
- The lines show the attention from each token (left) to every other token (right). Darker lines indicate higher attention weights. When multiple heads are selected, the attention weights are overlaid on one another. 


In [ ]:
display.display(display.HTML('<script src="/static/components/requirejs/require.js"></script>'))

head_view(attention, tokens, sentence_b_start, layer=0, heads=0)

# ParsBERT

In [ ]:
# version 2.0
# model_name_or_path = "HooshvareLab/bert-fa-base-uncased"

# version 3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForMaskedLM.from_pretrained(model_name_or_path, output_attentions=True)

In [ ]:
sentence_a = "حیوان از خیابان عبور نکرد."
sentence_b = "زیرا او خیلی خسته بود."


inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs["token_type_ids"]
input_ids = inputs["input_ids"]
attention = model(input_ids, token_type_ids=token_type_ids)[-1]

print(f"Attentoin has {len(attention)} heads")

sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist()
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
print("Tokens:", "\n".join(tokens))

Attentoin has 12 heads
Tokens: [CLS]
حیوان
از
خیابان
عبور
نکرد
.
[SEP]
زیرا
او
خیلی
خسته
بود
.
[SEP]


## Head View

The attention-head view visualizes attention in one or more heads in a particular layer in the model.


**Usage**

- **Hover** over any **token** on the left/right side of the visualization to filter attention from/to that token. The colors correspond to different attention heads.
- **Double-click** on any of the **colored tiles** at the top to filter to the corresponding attention head.
- **Single-click** on any of the **colored tiles** to toggle selection of the corresponding attention head. 
- **Click** on the **Layer** drop-down to change the model layer (zero-indexed).
- The lines show the attention from each token (left) to every other token (right). Darker lines indicate higher attention weights. When multiple heads are selected, the attention weights are overlaid on one another. 


In [ ]:
display.display(display.HTML('<script src="/static/components/requirejs/require.js"></script>'))

head_view(attention, tokens, sentence_b_start, layer=0, heads=0)